In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取 DeepSeek API Key
deepseek_api_key = os.environ['DEEPSEEK_API_KEY']

# 配置 OpenAI 客户端以兼容 DeepSeek API（需确认 DeepSeek 是否兼容 OpenAI 格式）
openai.api_key = deepseek_api_key
openai.base_url = "https://api.deepseek.com/v1"  # 根据 DeepSeek 文档调整

In [2]:
from langchain_openai import ChatOpenAI

# 修改后的 DeepSeek 配置
llm = ChatOpenAI(
    openai_api_key=os.getenv("DEEPSEEK_API_KEY"),  # 从环境变量读取 DeepSeek 的 Key
    base_url="https://api.deepseek.com/v1",        # 指向 DeepSeek 的 API 地址
    model_name="deepseek-reasoner",                   # 根据 DeepSeek 支持的模型名称修改
    temperature=0.0                              # 保持原有参数
)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001905C290200>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001905C291EE0>, root_client=<openai.OpenAI object at 0x000001905BEA2120>, root_async_client=<openai.AsyncOpenAI object at 0x000001905C22F4D0>, model_name='deepseek-reasoner', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.deepseek.com/v1')

In [3]:
from langchain.prompts.chat import ChatPromptTemplate

template = "你是一个翻译助手，可以帮助我将 {input_language} 翻译成 {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
messages  = chat_prompt.format_messages(input_language="中文", output_language="英文", text=text)
messages

[SystemMessage(content='你是一个翻译助手，可以帮助我将 中文 翻译成 英文.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。', additional_kwargs={}, response_metadata={})]

In [4]:
output  = llm.invoke(messages)
output

AIMessage(content='I carried luggage heavier than my body, swam down to the Nile\'s riverbed, passed through several bolts of lightning and saw clusters of halos - uncertain if this is the place.\n\n翻译解析：\n1. "比身体重的行李"译为"luggage heavier than my body"，保留了原文的物理重量与隐喻情感的双重含义\n2. "游入尼罗河底"处理为"swam down to the Nile\'s riverbed"，用介词down to强化纵深的空间感\n3. "闪电"选用"bolts of lightning"，突出闪电的瞬时性与穿透力\n4. "光圈"译为"halos"，既保留光圈的视觉效果，又暗示宗教/神秘主义色彩\n5. 破折号连接视觉奇观与心理状态，形成诗歌的节奏留白\n6. 整体采用自由诗体翻译，通过分行和标点还原原文的朦胧意境', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 598, 'prompt_tokens': 47, 'total_tokens': 645, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 410, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 47}, 'model_name': 'deepseek-reasoner', 'system_fingerprint': 'fp_5417b77867_prod0225', 'fin

In [5]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(output)

'I carried luggage heavier than my body, swam down to the Nile\'s riverbed, passed through several bolts of lightning and saw clusters of halos - uncertain if this is the place.\n\n翻译解析：\n1. "比身体重的行李"译为"luggage heavier than my body"，保留了原文的物理重量与隐喻情感的双重含义\n2. "游入尼罗河底"处理为"swam down to the Nile\'s riverbed"，用介词down to强化纵深的空间感\n3. "闪电"选用"bolts of lightning"，突出闪电的瞬时性与穿透力\n4. "光圈"译为"halos"，既保留光圈的视觉效果，又暗示宗教/神秘主义色彩\n5. 破折号连接视觉奇观与心理状态，形成诗歌的节奏留白\n6. 整体采用自由诗体翻译，通过分行和标点还原原文的朦胧意境'

In [6]:
chain = chat_prompt | llm | output_parser
chain.invoke({"input_language":"中文", "output_language":"英文","text": text})

'I carry luggage heavier than my body, diving into the depths of the Nile River. After passing through several bolts of lightning, I see clusters of halos - uncertain if this is the destined place.\n\n（注：译文在保持原有意境的同时，采用了诗意的语言处理：\n1. "比身体重的行李"译为"luggage heavier than my body"保留直译的震撼力\n2. "游入"选用"diving"强调主动潜入的动作\n3. "光圈"译为"halos"既保留光晕意象又带有神圣感\n4. 末句增补"destined"强化寻找归属地的迷茫感\n5. 通过分行和标点保留原文的呼吸节奏）'

In [7]:
text = 'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'
chain.invoke({"input_language":"英文", "output_language":"中文","text": text})

'我背着比身躯还要沉重的行李潜入尼罗河底，穿越几道闪电般的光芒后，眼前浮现出一堆光晕，不知是否就是那个地方。\n\n（说明：这个译文在保持原文意象的基础上进行了诗化处理。1. "heavier than my body"译为"比身躯还要沉重"，用"身躯"替代直白的"身体"更显文学性；2. "flashes of lightning"译为"闪电般的光芒"既保留闪电的意象又增加画面感；3. "a pile of halos"译为"一堆光晕"准确传递神秘感；4. 句式调整为流水句，用逗号营造出意识流般的沉浸感，最后用"不知"替代"不确定"更符合中文表达习惯，整体呈现出梦境般的超现实氛围。）'

In [14]:
import sys
sys.path.append(r'D:\My_Files\实验室学习相关\大模型部署') # 将父目录放入系统路径中

# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from langchain_community.embeddings import ZhipuAIEmbeddings

from langchain_chroma import Chroma

In [15]:
_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

In [20]:
# 定义 Embeddings
embedding = ZhipuAIEmbeddings(model="embedding-3")

# 向量数据库持久化路径
persist_directory = r'D:\My_Files\实验室学习相关\大模型部署\chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)

In [21]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：50


In [23]:
question="哪个农场种棉花"
docs = vectordb.similarity_search(question,k=5)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：5


In [24]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第0个内容: 
 Farm_ID: F039
Crop_Type: Cotton
Farm_Area(acres): 220.48
Irrigation_Type: Flood
Fertilizer_Used(tons): 9.96
Pesticide_Used(kg): 2.91
Yield(tons): 10.53
Soil_Type: Clay
Season: Zaid
Water_Usage(cubic meters): 82549.03
-----------------------------------------------------
检索到的第1个内容: 
 Farm_ID: F043
Crop_Type: Cotton
Farm_Area(acres): 78.79
Irrigation_Type: Flood
Fertilizer_Used(tons): 1.35
Pesticide_Used(kg): 3.0
Yield(tons): 11.45
Soil_Type: Sandy
Season: Zaid
Water_Usage(cubic meters): 94754.73
-----------------------------------------------------
检索到的第2个内容: 
 Farm_ID: F036
Crop_Type: Cotton
Farm_Area(acres): 446.16
Irrigation_Type: Manual
Fertilizer_Used(tons): 4.35
Pesticide_Used(kg): 3.47
Yield(tons): 12.53
Soil_Type: Loamy
Season: Zaid
Water_Usage(cubic meters): 38874.28
-----------------------------------------------------
检索到的第3个内容: 
 Farm_ID: F021
Crop_Type: Cotton
Farm_Area(acres): 377.05
Irrigation_Type: Drip
Fertilizer_Used(tons): 5.95
Pesticide_Used(kg): 0.91
Yi

In [26]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)

In [27]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

In [41]:
question_1 = "一共有多少个农场种植棉花？"
question_2 = "王阳明是谁？"

In [42]:
result = qa_chain.invoke({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
根据提供的上下文，共有4个农场种植棉花（F039、F043、F036、F001）。谢谢你的提问！


In [35]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
我不知道。谢谢你的提问！


In [36]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
llm.predict(prompt_template)

C:\Users\13967\AppData\Local\Temp\ipykernel_3056\2093528877.py:5: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict(prompt_template)


'**南瓜书**（《机器学习公式详解》）是一本专注于解析周志华教授《机器学习》（俗称“西瓜书”）中数学公式和难点的开源学习资料，由Datawhale团队编写。其核心内容如下：\n\n### 1. **定位与作用**\n- **西瓜书伴侣**：作为《机器学习》的补充，重点讲解书中复杂的数学推导与理论细节，降低学习门槛。\n- **公式拆解**：逐章逐条解析西瓜书中的公式，提供详细的数学证明和步骤演示，帮助读者理解背后的数学逻辑。\n\n### 2. **内容特点**\n- **覆盖全面**：涵盖线性模型、神经网络、支持向量机等关键章节的公式推导。\n- **可视化辅助**：通过图表和实例解释抽象概念（如梯度下降的几何意义）。\n- **误区标注**：标注常见理解错误，例如贝叶斯估计中先验/后验分布的混淆点。\n\n### 3. **适用人群**\n- **机器学习初学者**：适合数学基础薄弱的学习者补充微积分、概率论知识。\n- **进阶查漏补缺**：帮助已有基础的读者系统梳理理论体系，解决“一知半解”问题。\n\n### 4. **获取方式**\n- **开源免费**：通过GitHub或Datawhale官网获取PDF版本，支持社区协作更新。\n- **配套代码**：部分章节提供Python代码实现（如决策树分裂准则的实现示例）。\n\n### 5. **与其他资源的关联**\n- 与“西瓜书”形成“理论+推导”组合，与《机器学习实战》（“葫芦书”）构成“理论-推导-应用”学习链路。\n\n**示例**：在解析《机器学习》第6章支持向量机时，南瓜书用拉格朗日对偶性分步演示了间隔最大化问题的转化过程，并附上了KKT条件的几何解释图，帮助读者直观理解非线性分类问题的数学处理。\n\n如需深入学习，可直接访问南瓜书GitHub仓库下载最新版，结合西瓜书对应章节同步阅读。'

In [37]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
llm.invoke(prompt_template)

AIMessage(content='王阳明（1472—1529），名守仁，字伯安，号阳明，浙江余姚人，是明代著名的思想家、哲学家、军事家和教育家，陆王心学的集大成者。他的思想对东亚文化圈（尤其是中国、日本、朝鲜半岛）产生了深远影响。\n\n### 主要贡献与思想\n1. **心学体系**  \n   王阳明继承并发展了南宋陆九渊的“心学”，提出核心观点：  \n   - **心即理**：认为宇宙的法则（理）不在外部，而是根植于人的内心。  \n   - **知行合一**：反对“知与行分离”，强调真正的认知必须通过行动体现（例如：知道孝顺却不行孝，等于不知孝）。  \n   - **致良知**：主张通过内省和实践，唤醒人内心本有的道德判断力（良知），并以此指导行为。\n\n2. **事功成就**  \n   他不仅是思想家，也是能臣：  \n   - 平定江西匪患、宁王朱宸濠叛乱，展现卓越军事才能。  \n   - 推行基层治理，注重教化与民生结合。\n\n3. **教育理念**  \n   强调“人人皆可为圣人”，主张教育应启发本心，反对僵化的教条学习。\n\n### 重要经历\n- **龙场悟道**：因得罪宦官被贬贵州龙场驿，在困境中顿悟“心即理”，奠定心学基础。  \n- **讲学著述**：晚年广收门徒，代表作《传习录》由其弟子整理，系统阐述心学思想。\n\n### 历史地位\n王阳明突破程朱理学的框架，开创了儒学新方向，被称为“儒家四圣”之一（孔子、孟子、朱熹、王阳明）。他的思想在近代启发过梁启超、孙中山等人物，并在日本推动了明治维新前的思想变革。至今，“阳明心学”仍被视为注重实践与精神修养的重要哲学资源。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 824, 'prompt_tokens': 15, 'total_tokens': 839, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 409, 'rejected_prediction_toke

In [38]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

C:\Users\13967\AppData\Local\Temp\ipykernel_3056\2228008247.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [39]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
question = "我可以学习到关于提示工程的知识吗？"
result = qa({"question": question})
print(result['answer'])

根据提供的上下文数据，这些信息主要是关于不同农场的农业数据（如作物类型、灌溉方式、产量等），并没有涉及任何与提示工程（Prompt Engineering）相关的内容。因此，从这些数据中无法学习到关于提示工程的知识。

提示工程通常指设计和优化自然语言处理中的提示（Prompts）以提高模型生成结果的准确性和相关性，而这需要与AI模型交互的实践和理论学习。如果您有关于农业数据分析或其他相关领域的问题，我很乐意帮助解答！


In [40]:
question = "为什么这门课需要教这方面的知识？"
result = qa({"question": question})
print(result['answer'])

根据提供的上下文数据，其中主要涉及不同农场的农业相关信息（如作物类型、灌溉方式、产量等），但未提及任何与课程内容或提示工程（Prompt Engineering）相关的内容。因此，无法从给定数据中推断出“为什么这门课程需要教授提示工程的知识”这一问题的答案。

如果您有其他上下文或具体问题需要结合当前数据进行分析，请提供更多信息。
